In [137]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('results') if isfile(join('results', f))]

In [138]:
result_files = []
for file in onlyfiles:
    try:
        if file.count('_') >= 3:
            method = [m for m in ['macbf', 'ddpg', 'v0', 'v49', 'v125'] if m in file][0]
            method = method.upper() if method in ['macbf', 'ddpg'] else 'CAM' if 'no_decompose' not in file else 'CAM_no_decompose'
            environment = [env for env in ['dubins_car', 'drone', 'multi_dynamic_dubins'] if env in file][0]
            density = int(file.split('_')[-1].replace('.pkl', ''))
            mode = [m for m in ['static', 'mixed', 'dynamic'] if m in file][0]
            print(f'{file},{environment},{mode},{method},{density}')
            result_files.append([file,environment,mode,method,density])
        else:
            continue
    except:
        continue

drone_v49_mixed_8.pkl,drone,mixed,CAM,8
dubins_car_ddpg_v8_dynamic_16.pkl,dubins_car,dynamic,DDPG,16
drone_v49_no_decompose_mixed_32.pkl,drone,mixed,CAM_no_decompose,32
dubins_car_v125_dynamic_8.pkl,dubins_car,dynamic,CAM,8
dubins_car_macbf_v2_static_5.pkl,dubins_car,static,MACBF,5
multi_dynamic_dubins_macbf_v0_mixed_512.pkl,multi_dynamic_dubins,mixed,MACBF,512
drone_macbf_v2_mixed_512.pkl,drone,mixed,MACBF,512
multi_dynamic_dubins_ddpg_v0_mixed_256.pkl,multi_dynamic_dubins,mixed,DDPG,256
drone_v49_dynamic_3.pkl,drone,dynamic,CAM,3
multi_dynamic_dubins_v0_mixed_512.pkl,multi_dynamic_dubins,mixed,CAM,512
dubins_car_v125_dynamic_3.pkl,dubins_car,dynamic,CAM,3
multi_dynamic_dubins_v0_mixed_8.pkl,multi_dynamic_dubins,mixed,CAM,8
multi_dynamic_dubins_ddpg_v0_mixed_16.pkl,multi_dynamic_dubins,mixed,DDPG,16
drone_v49_dynamic_8.pkl,drone,dynamic,CAM,8
drone_macbf_v2_dynamic_3.pkl,drone,dynamic,MACBF,3
drone_ddpg_v9_static_0.pkl,drone,static,DDPG,0
drone_ddpg_v9_mixed_3.pkl,drone,mixed,DDPG,3
d

In [139]:
def get_map_size(mode, density):
    if mode=='static':
        size = 4
    if mode=='dynamic':
        size = int((density*16)**0.5)
    if mode=='mixed':
        size = 32
    return size

def get_agent_num(mode, density):
    if mode=='static':
        num_agents = 3
    if mode=='dynamic':
        num_agents = density
    if mode=='mixed':
        num_agents = density
    return num_agents

In [140]:
# save the environment,dataset,method,density,collision_per_env,collision_per_agent,success_rate,path_length

def get_reward(num_collides, num_success, path_length, agent_num):
    return (num_collides*(-1) + num_success*10)/agent_num + -1e-2*path_length

def get_safe_rate(num_collides, path_length, agent_num):
    return 1-(num_collides/(path_length*agent_num))

In [141]:
import seaborn as sns
sns.set_style("dark")
total_results = sns.load_dataset("/results", data_home='./results')

In [142]:
import pickle as pkl
import numpy as np

environment = 'multi_dynamic_dubins'

result_files = [r for r in result_files if r[1]==environment]
for file,environment,mode,method,density in result_files:
    
    if environment=='drone':
        from drone_v34 import *
    elif environment=='dubins_car':
        from v109 import *    
    elif environment=='multi_dynamic_dubins':
        from v0_multi_dynamic_dubins import *
    
    # environment,dataset,method,density
    if ((total_results['dataset']==mode)&(total_results['density']==density)&
        (total_results['environment']==environment)&(total_results['method']==method)).any():
        continue
    
    with open(f'dataset/{environment}_{mode}_{density}.pkl', 'rb') as f:
        valid_dataset = pkl.load(f)
        
    with open(f'results/{file}', 'rb') as f:
        result_dict = pkl.load(f)
        paths = result_dict['paths']
    
    map_size = get_map_size(mode, density)
    num_agents = get_agent_num(mode, density)
    env = create_env(num_agents=num_agents, size=map_size, density=0, min_dist=0)

    # reproduce the trajectory
    rewards = []
    safe_rates = []
    for param, path in zip(valid_dataset, paths):
        env.world.obstacles, env.world.agent_goals, env.world.agents = deepcopy(param)
        num_collides = 0
        num_success = np.array([False]*num_agents)
        for timestep in path:
            env.world.agents = timestep
            status = env.world.get_status()
            num_collides += sum(['danger' in s for s in status])
            num_success = num_success | np.array(['done' in s for s in status])
        rewards.append(get_reward(num_collides, num_success.sum(), len(path), num_agents))
        safe_rates.append(get_safe_rate(num_collides, len(path), num_agents))
    print(f'{environment},{mode},{method},{density},{np.mean(safe_rates)},{np.mean(rewards)},{np.std(safe_rates)},{np.std(rewards)}')

multi_dynamic_dubins,mixed,DDPG,512,0.9916205607334309,0.016269531250000142,0.0034186398040855427,1.7784507604844313
